# Какво представлява производната?
Производна на функцията $f(x)$ може да бъде дефинирана по следния начин:

$$
f'(x) = \lim _{\Delta x \to 0} \frac {f(x + \Delta x) - f(x)} {\Delta x}
$$

### Приближения на производна
Тази дефиниция не може да се реализира в компютърната аритметика, тъй като паметта на всеки компютър е ограничена и това налага определени лимитации от гледна точка на броя значещи цифри след десетичната запетая, с които се представя дадено реално число. Следователно ние можем да направим следното приближение:

$$
f'(x) \approx \frac {f(x + \Delta x) - f(x)} {\Delta x}
$$

Ако формално положим $\Delta x = - \Delta y$, то ще получим друго подобно приближение:
$$
f'(x) \approx \frac {f(x + \Delta x) - f(x)} {\Delta x} = \frac {f(x - \Delta y) - f(x)} {- \Delta y} = \frac {f(x) - f(x - \Delta y)} {\Delta y}
$$
$$
\implies f'(x) \approx \frac {f(x) - f(x - \Delta x)} {\Delta x}
$$

Ако съберем двата израза ще получим:
$$
f'(x) \approx \frac {f(x + \Delta x) - f(x - \Delta x)} {2 \Delta x}
$$

### Визуализация на методите за приближение на производна
В следващите няколко графики може да видите разликите в точността и да сравните грешката, която се допуска в следствие на направеното приближение, в зависимост от използвания метод и стойността на $\Delta x$. Ще работим с функцията $f(x) = e^x$, чиято производна има същия вид ($f'(x) = e^x$).

In [1]:
from typing import List, Callable
from scipy.misc import derivative
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nodes = np.arange(-5, 5, 0.01)


def f1(num: int) -> float:
    return np.exp(num)


def approximated_derivation_1(num: float, delta: float) -> float:
    return (np.exp(num + delta) - np.exp(num)) / delta


def approximated_derivation_2(num: float, delta: float) -> float:
    return (np.exp(num) - np.exp(num - delta)) / delta


def approximated_derivation_3(num: float, delta: float) -> float:
    return (np.exp(num + delta) - np.exp(num - delta)) / (2 * delta)


def print_derivation_info(deltas: List[int], derivations: List[Callable[[float, float], float]]) -> None:
    for i in range(len(derivations)):
        current_derivation = derivations[i]

        for j in range(len(deltas)):
            current_delta = deltas[j]

            plt.title(f'Approximated derivation #{i + 1} with delta {current_delta}')
            plt.xlabel('x')
            plt.ylabel("f'(x)")
            plt.plot(nodes, [f1(p) for p in nodes], nodes, [current_derivation(p, current_delta) for p in nodes], 'r')
            plt.savefig(f"plots/approx_deriv_{i + 1}_{j + 1}.png")
            plt.close()


all_deltas = [0.5, 10 ** -3, 10 ** -15]
all_derivations = [approximated_derivation_1, approximated_derivation_2, approximated_derivation_3]
print_derivation_info(all_deltas, all_derivations)


![Approximated derivation #1 with delta 0.5](./plots/approx_deriv_1_1.png)
![Approximated derivation #1 with delta 0.001](./plots/approx_deriv_1_2.png)
![Approximated derivation #1 with delta 1e-15](./plots/approx_deriv_1_3.png)
![Approximated derivation #2 with delta 0.5](./plots/approx_deriv_2_1.png)
![Approximated derivation #2 with delta 0.001](./plots/approx_deriv_2_2.png)
![Approximated derivation #2 with delta 1e-15](./plots/approx_deriv_2_3.png)
![Approximated derivation #3 with delta 0.5](./plots/approx_deriv_3_1.png)
![Approximated derivation #3 with delta 0.001](./plots/approx_deriv_3_2.png)
![Approximated derivation #3 with delta 1e-15](./plots/approx_deriv_3_3.png)

### Анализиране на резултатите

От дефиницията за производна следва, че при намаляване на $\Delta x$ приближението, което получаваме, трябва да бъде по-добро (от дефиницията имаме $\Delta x \to 0$). И наистина - намаляването на грешката се вижда ясно в първите две графики за всеки един от методите на приближение на производната, които сме използвали. При $\Delta x = 10^{-15}$ обаче получената апроксимация не е добра.

Този експеримент нагледно показва това, което бе описано по-рано - тъй като реалните числа в паметта на компютъра се представят с ограничен брой значещи цифри, грешката първоначално ще намалява при намаляване на стъпката, но когато бъде достигнат максимума на машинната точност (и следователно намаляването на стъпката няма повече да подобрява точността на същата тази апроксимация), грешката ще започне да расте заради увеличаването броя на операциите. Това е причината за появата на "шума" на третата графика за всеки един от методите на приближение на производната, които сме използвали.

# Какво представлява численото диференциране?
Численото диференциране ни помага да решаваме практически задачи от този вид: _"Да се намери производна на функцията $f$ в определена точка, като стойностите на функцията са известни само в краен брой точки."_ Също така техниките на численото диференциране са приложими и за функции със сложен аналитичен характер, за които не е оправдано точното пресмятане на производна.

Обикновено решението на подобни задачи представлява намирането на производна на някоя приближаваща функция (като за целта могат да бъдат използвани и интерполационните полиноми. Това, което трябва да се вземе предвид обаче, е, че дори малки изменения на приближаващата функция могат да доведат до по-големи изменения в производната. Нека разгледаме конкретен пример:

$$
f(x) - P(x) = -10^{-5}\sin(mx)
$$

Тук отклонението на приближаващия полином $P(x)$ спрямо функцията $f(x)$ е $R(x) = -10^{-5}\sin(mx)$. Очевидно е, че $|R(x)| \leq 10^{-5}$. Нека обаче да видим какво ще се случи, ако диференцираме двете части на това уравнение:

$$
f'(x)-P'(x) = -m10^{-5}\cos(mx)
$$

Този път ясно се вижда, че грешката $R'(x) = -m10^{-5}\cos(mx)$ може да бъде произволно голяма в зависимост от избраната стойност за параметъра $m$. Следва да заключим, че задачата за числено диференциране не е устойчива.

### Числено диференциране чрез _Интерполационния полином на Нютон_

Нека въведем следните означения:
$$
\omega_n(x) = (x - x_0)(x - x_1)\dots(x - x_n)
$$
$$
f_{[x_i,x_{i+1},\dots,x_{i+k+1}]} = \frac {f_{[x_{i+1},x_{i+2},\dots,x_{i+k+1}]} - f_{[x_i,x_{i+1},\dots,x_{i+k}]}} {x_{i+k+1}-x_{i}}
$$

Производната на функцията $\omega_n$ има едно чудесно свойство, което ще използваме в някои означения по-напред:
$$
\omega_n'(x) = (x - x_1)(x - x_2)\dots(x - x_n) + (x - x_0)(x - x_2)\dots(x - x_n) + \dots + (x - x_0)(x - x_1)\dots(x - x_{n-1}) = \sum_{i=0}^{n} \prod_{j \neq i}^{n} (x - x_j)
$$
$$
\omega_n'(x_k) = (x_k - x_0)(x_k - x_1)\dots(x_k - x_{k-1})(x_k - x_{k+1})\dots(x_k - x_n)
$$

Тогава от формулата за представяне на грешка с разделена разлика получаваме:
$$
f(x) = N_n(f;x) + f_{[x_0,x_1,\dots,x_n,x]}\omega_n(x)
$$
$$
f'(x) = N_n'(f;x) + \frac {df_{[x_0,x_1,\dots,x_n,x]}} {dx} \omega_n(x) + f_{[x_0,x_1,\dots,x_n,x]} \omega_n'(x)
$$

От дефиницията за производна знаем, че:
$$
\frac {df_{[x_0,x_1,\dots,x_n,x]}} {dx} = \lim_{\Delta x \to 0} \frac {f_{[x_0,x_1,\dots,x_n,x+\Delta x]} - f_{[x_0,x_1,\dots,x_n,x]}} {\Delta x}
$$

За да продължим с по-нататъшното решение на този израз, е необходимо да отбележим едно свойство на разделените разлики, а именно - симетричността им. Ако имаме пермутацията $\sigma : {0,\dots,n} \rightarrow {0,\dots,n}$, то $f_{[x_0,x_1,\dots,x_n]} = f_{[x_{\sigma(0)},x_{\sigma(1)},\dots,x_{\sigma(n)}]}$. Сега можем да се върнем към предишния израз:
$$
\lim_{\Delta x \to 0} \frac {f_{[x_0,x_1,\dots,x_n,x+\Delta x]} - f_{[x_0,x_1,\dots,x_n,x]}} {\Delta x} = \lim_{\Delta x \to 0} f_{[x_0,x_1,\dots,x_n,x,x + \Delta x]} = f_{[x_0,x_1,\dots,x_n,x,x]}
$$

След като знаем как се диференцират разделени разлики, нека се върнем обратно към изчисляването на производната за $f$:
$$
f'(x) = N_n'(f;x) + f_{[x_0,x_1,\dots,x_n,x,x]} \omega_n(x) + f_{[x_0,x_1,\dots,x_n,x]} \omega_n'(x)
$$

Освен това, ако $f \in C^{n+2}[a,b], a \le x_o < x_1 < \dots < x_n \le b, x \in [a,b]$, то тогава можем да опростим записа, като представим разделената разлика чрез производна на $f$ в дадена точка:
$$
f_{[x_0,x_1,\dots,x_k]} = \frac {f^{(k)}(\xi)} {k!}
$$
$$
\implies f'(x) = N_n'(f;x) + \frac {f^{(n+2)}(\xi)} {(n+2)!} \omega_n(x) + \frac {f^{(n+1)}(\eta)} {(n+1)!} \omega_n'(x),
$$
където $\xi$ и $\eta$ са точки от интервала $(a, b)$.

От това уравнение можем да заключим, че грешката при приближаване на $f'(x)$ с $N_n'(f;x)$ е:
$$
R'(x) = \frac {f^{(n+2)}(\xi)} {(n+2)!} \omega_n(x) + \frac {f^{(n+1)}(\eta)} {(n+1)!} \omega_n'(x)
$$

Ако направим следните означение - $M_{n+2} = max_{x \in [a,b]}|f^{(n+2)}(x)|$ и $M_{n+1} = max_{x \in [a,b]}|f^{(n+1)}(x)|$, окончателно получаваме:
$$
|R'(x)| = \frac {|f^{(n+2)}(\xi)|} {(n+2)!} |\omega_n(x)| + \frac {|f^{(n+1)}(\eta)|} {(n+1)!} |\omega_n'(x)| \le \frac {M_{n+2}} {(n + 2)!} |\omega_n(x)| + \frac {M_{n+1}} {(n + 1)!} |\omega_n'(x)|
$$

### Формули за производна на _Интерполационния полином на Нютон_

$$
N_n(f; x) = f(x_0) + f_{[x_0, x_1]}(x - x_0) + f_{[x_0, x_1, x_2]}(x - x_0)(x - x_1) + \dots + f_{[x_0, x_1, \dots, x_n]}(x - x_0)(x - x_1) \dots (x - x_{n-1})
$$
$$
N_n'(f; x) = f_{[x_0, x_1]} + f_{[x_0, x_1, x_2]}[(x - x_0) + (x - x_1)] + f_{[x_0, x_1, x_2, x_3]}[(x - x_0)(x - x_1) + (x - x_0)(x - x_2) + (x - x_1)(x - x_2)] + \dots + f_{[x_0, x_1, \dots, x_n]} \sum_{i=0}^{n-1} \prod_{j \neq i}^{n-1} (x - x_j)
$$

Този израз е доста комплексен, но лесно може да покажем, че той ще се опрости, когато $x$ приема за стойност някой от интерполационните възли. Нека разгледаме например случая, когато $x = x_0$:
$$
N_n'(f; x) = f_{[x_0, x_1]} + f_{[x_0, x_1, x_2]}(x_0 - x_1) + f_{[x_0, x_1, x_2, x_3]}(x_0 - x_1)(x_0 - x_2) + \dots + f_{[x_0, x_1, \dots, x_n]}(x_0 - x_1)(x_0 - x_2)\dots(x_0 - x_{n-1})
$$

При $\omega'(x) = 0$ изразът също би се опростил, а това се получава, когато имаме четен брой възли, които са симетрично разположени около точката $x$:
$$
x - x_i = x_{n-i} - x, i = 0, 1, \dots, \frac {n - 1} {2}
$$

В повечето случай, когато става въпрос за решаване на задачи, най-често се използват частни случаи на тази формула:
1. При $n = 1$ и възли $x_0 = a, x_1 = a + h$ получаваме $N'(f; a) = f_{[a, a + h]} = \frac {f(a + h) - f(a)} {h}$, а допуснатата грешка е $R'(a) = \frac {f''(\eta)} {2} h, \eta \in (a, a + h)$.
2. При $n = 1$ и възли $x_0 = a - h, x_1 = a + h$ получаваме $N'(f; a) = f_{[a - h, a + h]} = \frac {f(a + h) - f(a - h)} {2h}$, а допуснатата грешка е $R'(a) = -\frac {f'''(\xi)} {6} h^2, \xi \in (a - h, a + h)$.
3. При $n = 2$ и възли $x_0 = a, x_1 = a + h, x_2 = a + 2h$ получаваме $N'(f; a) = f_{[a, a + h]} + f_{[a, a + h, a + 2h]}(-h) = \frac {-3f(a) + 4f(a + h) -f(a + 2h)} {2h}$, а допуснатата грешка е $R'(a) = -\frac {f'''(\eta)} {3} h^2, \eta \in (a, a + 2h)$.
4. При $n = 2$ и възли $x_0 = a, x_1 = a + h, x_2 = a + 2h$ получаваме $N'(f; a + 2h) = \frac {f(a) - 4f(a + h) + 3f(a + 2h)} {2h}$, а допуснатата грешка е $R'(a + 2h) = -\frac {f'''(\eta)} {3} h^2, \eta \in (a, a + 2h)$.
5. При $n = 2$ и възли $x_0 = a - h, x_1 = a, x_2 = a + h$ получаваме същата формула от точка 2.

#### Доказателство

Нека $n = 1$, $x_0 = a$, $x_1 = a + h$, $x = a$
$$
N_1(f; x) = f_{[x_0]} + f_{[x_0, x_1]}(x - x_0) = f_{[a]} + f_{[a, a + h]}(x - a)
$$

Когато диференцираме това уравнение спрямо $x$ ще получим това, което записахме по-горе в обобщената формула:
$$
N_1'(f; x) = f_{[a, a + h]} = \frac {f(a + h) - f(a)} {h}
$$

Нека сега да изведем и формулата за грешка:
$$
\omega(x) = (x - a)(x - a - h) \implies \omega(a) = 0
$$
$$
\omega'(x) = 2(x - a) - h \implies \omega'(a) = -h
$$
$$
|f'(a) - \frac {f(a + h) - f(a)} {h}| \le \frac {M_2} {2} |\omega'(a)| = \frac {M_2} {2} h
$$

---

Нека $n = 1$, $x_0 = a - h$, $x_1 = a + h$, $x = a$
$$
N_1(f; x) = f_{[x_0]} + f_{[x_0, x_1]}(x - x_0) = f_{[a - h]} + f_{[a - h, a + h]}(x - a + h)
$$

Когато диференцираме това уравнение спрямо $x$ ще получим това, което записахме по-горе в обобщената формула:
$$
N_1'(f; x) = f_{[a - h, a + h]} = \frac {f(a + h) - f(a - h)} {2h}
$$

Нека сега да изведем и формулата за грешка:
$$
\omega(x) = (x - (a - h))(x - (a + h)) = (x - a)^2 - h^2 \implies \omega(a) = -h^2
$$
$$
\omega'(x) = 2(x - a) \implies \omega'(a) = 0
$$
$$
|f'(a) - \frac {f(a + h) - f(a - h)} {2h}| \le \frac {M_3} {3!} |\omega'(a)| = \frac {M_3} {6} h^2
$$

---

Доказателството на останалите формули е аналогично.

### Формули за пресмятане на втори производни при три равноотстоящи възли
Нека приемем, че $x_0 = a, x_1 = a + h, x_2 = a + 2h$. Тогава можем да изведем следните формули:
$$
f''(a) = \frac {1} {2h} [f(a) - 2f(a + h) + f(a + 2h)] - hf'''(\xi) + \frac {h^2} {6} f^{(4)}(\eta)
$$
$$
f''(a + h) = \frac {1} {2h} [f(a) - 2f(a + h) + f(a + 2h)] - \frac {h^2} {12} f^{(4)}(\xi)
$$
$$
f''(a) = \frac {1} {2h} [f(a) - 2f(a + h) + f(a + 2h)] + hf'''(\xi) - \frac {h^2} {6} f^{(4)}(\eta)
$$
$$
\xi, \eta \in [a, a + 2h]
$$

In [3]:
# This cell will contain helper functions used along the way.
def extract_single_dimension(data: List[tuple[float, float]], dimension: int) -> List[float]:
    return [d[dimension] for d in data]


def calculate_divided_differences(data: List[tuple[float, float]]) -> List[List[float]]:
    ans = [[] for _ in range(len(data))]
    for i in range(len(data)):
        ans[0].append(data[i][1])

    for k in range(len(data) - 1):
        for i in range(len(ans[k]) - 1):
            numerator = ans[k][i + 1] - ans[k][i]
            denominator = data[i + k + 1][0] - data[i][0]

            if numerator == 0 or denominator == 0:
                ans[k + 1].append(0)
            else:
                ans[k + 1].append(numerator / denominator)

    return ans


def generate_func_data(x_points: List[float], f: Callable[[float], float]) -> List[tuple[float, float]]:
    ans = []

    for i in range(len(x_points)):
        ans.append((x_points[i], f(x_points[i])))

    return ans


def prepare_plot(title: str, x_label: str, y_label: str) -> None:
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.grid(axis='both')


def print_divided_differences(divided_differences: List[List[float]]) -> None:
    for i in range(1, len(divided_differences)):
        print()
        print(f'#{i + 1}')
        for j in range(len(divided_differences[i])):
            notation = ", ".join([f'x[{p}]' for p in range(j, i + j + 1)])
            print(f'f[{notation}] = {divided_differences[i][j]}')


def print_interpolation_nodes(data: List[tuple[float, float]]) -> None:
    for i in range(len(data)):
        print(f'x[{i}] = {data[i][0]}, f(x[{i}]) = {data[i][1]}')


def calculate_chebyshev_nodes(n: int, a: float, b: float) -> List[float]:
    ans = []

    for i in range(n):
        current_node = 0.5 * (a + b) + 0.5 * (b - a) * np.cos(((2 * i + 1) / (2 * n)) * np.pi)
        ans.append(current_node)

    return ans


def interpolate_newton(x: float, data: List[tuple[float, float]], divided_differences: List[List[float]]) -> float:
    ans = 0

    for i in range(len(divided_differences)):
        c = divided_differences[i][0]

        for j in range(i):
            c *= x - data[j][0]

        ans += c

    return ans


def visualize_newton_interpolation(name: str, data: List[tuple[float, float]], f: Callable[[float], float] = None,
                                   f_deriv: Callable[[float], float] = None, step: float = 0.01,
                                   is_verbose: bool = False) \
        -> tuple[Callable[[float], float], Callable[[float], float]]:
    if is_verbose:
        print_interpolation_nodes(data)

    divided_differences = calculate_divided_differences(data)
    if is_verbose:
        print_divided_differences(divided_differences)

    x_coordinates = extract_single_dimension(data, 0)
    y_coordinates = extract_single_dimension(data, 1)

    def interpolate_in_point(p: float) -> float:
        return interpolate_newton(p, data, divided_differences)

    def approximate_derivative_in_point(p: float) -> float:
        return derivative(interpolate_in_point, p, dx=10 ** -5)

    all_points_in_range = np.arange(min(x_coordinates), max(x_coordinates) + step, step)

    interpolation_values = [interpolate_in_point(p) for p in all_points_in_range]
    interpolation_errors = None

    interpolation_deriv_values = [approximate_derivative_in_point(p) for p in all_points_in_range]
    interpolation_deriv_errors = None

    prepare_plot('Graph of the interpolated function', 'x', 'f(x)')
    if f:
        f_values = [f(p) for p in all_points_in_range]
        interpolation_errors = [v1 - v2 for v1, v2 in zip(f_values, interpolation_values)]
        plt.plot(all_points_in_range, f_values, color='orange', linewidth=2, label="Original")

    plt.plot(all_points_in_range, interpolation_values, label="Interpolated")
    plt.plot(x_coordinates, y_coordinates, marker='o', markersize=3, linewidth=0, color='r',
             label="Interpolation nodes")

    plt.legend()
    plt.savefig(f"plots/{name}_interpolated_func.png")
    plt.close()

    if interpolation_errors is not None:
        prepare_plot('Graph of the interpolation error', 'x', 'R(x)')
        plt.plot(all_points_in_range, interpolation_errors, linewidth=1)
        plt.savefig(f"plots/{name}_interpolated_func_error.png")
        plt.close()

    prepare_plot('Graph of the interpolated function\'s derivative', 'x', 'f\'(x)')
    if f_deriv:
        f_deriv_values = [f_deriv(p) for p in all_points_in_range]
        interpolation_deriv_errors = [v1 - v2 for v1, v2 in zip(f_deriv_values, interpolation_deriv_values)]
        plt.plot(all_points_in_range, f_deriv_values, color='orange', linewidth=2, label="Original")

    plt.plot(all_points_in_range, interpolation_deriv_values, label="Interpolated")

    plt.legend()
    plt.savefig(f"plots/{name}_interpolated_deriv.png")
    plt.close()

    if interpolation_errors is not None:
        prepare_plot('Graph of the interpolation derivative error', 'x', 'R\'(x)')
        plt.plot(all_points_in_range, interpolation_deriv_errors, linewidth=1)
        plt.savefig(f"plots/{name}_interpolated_deriv_error.png")
        plt.close()

    return interpolate_in_point, approximate_derivative_in_point

# Примери

В следващите редове ще представя няколко примера, които да можем да разгледаме и да анализираме заедно.

### Диференциране на таблично зададена функция

Нека разгледаме таблично зададената функция $f$:

| x   | y   |
|-----|-----|
| -2  | 29  |
| -1  | -6  |
| 1   | -4  |
| 2   | -3  |
| 3   | 14  |

Тъй като не знаем нищо повече за тази функция, ние не можем да изчислим грешката на интерполиране, но все пак можем да си построим интерполационния полином и да апроксимираме нейната производна.

In [4]:
_ = visualize_newton_interpolation("example_1", [(-2, 29), (-1, -6), (1, -4), (2, -3), (3, 14)], is_verbose=True)

x[0] = -2, f(x[0]) = 29
x[1] = -1, f(x[1]) = -6
x[2] = 1, f(x[2]) = -4
x[3] = 2, f(x[3]) = -3
x[4] = 3, f(x[4]) = 14

#2
f[x[0], x[1]] = -35.0
f[x[1], x[2]] = 1.0
f[x[2], x[3]] = 1.0
f[x[3], x[4]] = 17.0

#3
f[x[0], x[1], x[2]] = 12.0
f[x[1], x[2], x[3]] = 0
f[x[2], x[3], x[4]] = 8.0

#4
f[x[0], x[1], x[2], x[3]] = -3.0
f[x[1], x[2], x[3], x[4]] = 2.0

#5
f[x[0], x[1], x[2], x[3], x[4]] = 1.0


![Graph of the interpolated function](./plots/example_1_interpolated_func.png)
![Graph of the interpolated function's derivative](./plots/example_1_interpolated_deriv.png)

### Анализиране на грешката при диференциране

Да започнем с един изключително лесен пример: $f(x) = sin(x) + cos(x)$. Примерният код ще построи Интерполационен полином на Нютон от 10-та степен, а за интерполационни възли се използват нулите на Полинома на Чебишов в диапазона $[0, 10]$. От получените резултати ще забележим, че грешката при интерполиране на $f(x)$ и грешката при приближаване на $f'(x)$ са по-скоро минимални.

In [5]:
def f2(x: float):
    return np.sin(x) + np.cos(x)


def f2_deriv(x: float):
    return np.cos(x) - np.sin(x)


f2_data = generate_func_data(calculate_chebyshev_nodes(11, 0, 10), f2)
_ = visualize_newton_interpolation("example_2", f2_data, f2, f2_deriv)

![Graph of the interpolated function](./plots/example_2_interpolated_func.png)
![Graph of the interpolation error](./plots/example_2_interpolated_func_error.png)
![Graph of the interpolated function's derivative](./plots/example_2_interpolated_deriv.png)
![Graph of the interpolation derivative error](./plots/example_2_interpolated_deriv_error.png)

А какво би се случило, ако интервала, в който интерполираме съответната функция, е по-голям? Нека да видим!

Ще променим интервала, в който са дефинирани интерполационните ни възли на $[0, 100]$ (но забележете, че няма да променим броя на възлите).
Разбира се, след като се разгледат графиките по-долу, е очевидно, че точността на построения интерполационен полином е доста по-ниска спрямо това, което наблядавахме по-рано. И докато тогава заключихме, че грешката е по-скоро минимална, то тук виждаме, че тя приема доста по-сериозни стойности.

In [6]:
f2_data_wide = generate_func_data(calculate_chebyshev_nodes(11, 0, 100), f2)
_ = visualize_newton_interpolation("example_3", f2_data_wide, f2, f2_deriv)

![Graph of the interpolated function](./plots/example_3_interpolated_func.png)
![Graph of the interpolation error](./plots/example_3_interpolated_func_error.png)
![Graph of the interpolated function's derivative](plots/example_3_interpolated_deriv.png)
![Graph of the interpolation derivative error](plots/example_3_interpolated_deriv_error.png)

Следващият пример цели да демонстрира нагледно едно твърдение, което е записано по-горе, а именно, че задачата за числено диференциране не е устойчива. Ще разгледаме следната функция: $f(x) = sin(2x)cos(x) + sin(x)cos(2x)$. Примерният код ще построи Интерполационен полином на Нютон от 15-та степен, а за интерполационни възли се използват нулите на Полинома на Чебишов в диапазона $[0, 10]$. От получените резултати ще забележим, че грешката при интерполиране на $f(x)$ е приемлива, но грешката при приближаване на $f'(x)$ е много по-значителна.

Увеличаването на степента на интерполационния полином, който следва да бъде построен, ще намали грешките, но тази неустойчивост при апроксимацията на производната, ще се запази.

In [7]:
def f3(x: float):
    return np.sin(2 * x) * np.cos(x) + np.sin(x) * np.cos(2 * x)


def f3_deriv(x: float):
    return 3 * np.cos(x) * np.cos(2 * x) - 3 * np.sin(x) * np.sin(2 * x)


f3_data = generate_func_data(calculate_chebyshev_nodes(16, 0, 10), f3)
_ = visualize_newton_interpolation("example_4", f3_data, f3, f3_deriv)

![Graph of the interpolated function](./plots/example_4_interpolated_func.png)
![Graph of the interpolation error](./plots/example_4_interpolated_func_error.png)
![Graph of the interpolated function's derivative](./plots/example_4_interpolated_deriv.png)
![Graph of the interpolation derivative error](./plots/example_4_interpolated_deriv_error.png)

Нека да покажем и резултатите при повишаване на степента на полинома от 15-та на 17-та:

In [8]:
f3_data_better = generate_func_data(calculate_chebyshev_nodes(18, 0, 10), f3)
_ = visualize_newton_interpolation("example_5", f3_data_better, f3, f3_deriv)

![Graph of the interpolated function](./plots/example_5_interpolated_func.png)
![Graph of the interpolation error](./plots/example_5_interpolated_func_error.png)
![Graph of the interpolated function's derivative](./plots/example_5_interpolated_deriv.png)
![Graph of the interpolation derivative error](./plots/example_5_interpolated_deriv_error.png)

### Решаване на задачи

Нека разгледаме следната таблично зададена функция:

| x   | f(x) |
|-----|------|
| -1  | 1    |
| 1   | 2    |
| 2   | 1    |
| 3   | 0    |
| 4   | 4    |
| 5   | 3    |

Търсят се приближените стойности на първата производна на зададената функция в точките -1, 1 и 4, като се използват три точки, в които функията е вече известна. Също така трябва да оценим грешката, която се допуска, ако $|f'''(x)| \le \frac {1} {10}$ в интервала $(-1, 5)$.

#### Решение

За да намерим приближената стойност на първата производна в точка -1, можем да използваме следните точки - $(-1, 1, 3)$ или $(-1, 2, 5)$. По-подходящо би било да изберем първата тройка, тъй като при нея стъпката ($h = 2$) е по-малка и това би довело до по-добра точност.

$$N'(f; -1) = \frac {-3f(-1) + 4f(1) -f(3)} {4} = \frac {-3 * 1 + 4 * 2 - 0} {4} = \frac {5} {4}$$
$$R'(-1) = -4 \frac {f'''(\eta)} {3} \implies |R'(-1)| \le \frac {4} {30}$$

In [9]:
(_, solve_problem_1_1) = visualize_newton_interpolation("problem_1", [(-1, 1), (1, 2), (3, 0)])
print(f"The approximated value is {solve_problem_1_1(-1)}")

The approximated value is 1.250000000002638


![Graph of the interpolated function](./plots/problem_1_interpolated_func.png)
![Graph of the interpolated function's derivative](./plots/problem_1_interpolated_deriv.png)

---

За да намерим приближената стойност на първата производна в точка 1, можем да използваме следните точки - $(1, 2, 3)$ или $(1, 3, 5)$. По-подходящо би било да изберем първата тройка, тъй като при нея стъпката ($h = 1$) е по-малка и това би довело до по-добра точност.

$$N'(f; 1) = \frac {-3f(1) + 4f(2) -f(3)} {2} = \frac {-3 * 2 + 4 * 1 - 0} {2} = -1$$
$$R'(1) = -\frac {f'''(\eta)} {3} \implies |R'(1)| \le \frac {1} {30}$$

In [10]:
(_, solve_problem_1_2) = visualize_newton_interpolation("problem_2", [(1, 2), (2, 1), (3, 0)])
print(f"The approximated value is {solve_problem_1_2(1)}")

The approximated value is -1.0000000000065512


![Graph of the interpolated function](./plots/problem_2_interpolated_func.png)
![Graph of the interpolated function's derivative](./plots/problem_2_interpolated_deriv.png)

---

За да намерим приближената стойност на първата производна в точка 4, нека разгледаме тройката $(2, 3, 4)$. При нея стъпката $h = 1$.

$$N'(f; 4) = \frac {f(2) - 4f(3) + 3f(4)} {2} = \frac {1 - 4 * 0 + 3 * 4} {2} = \frac {13} {2}$$
$$R'(4) = -\frac {f'''(\eta)} {3} \implies |R'(4)| \le \frac {1} {30}$$

In [11]:
(_, solve_problem_1_3) = visualize_newton_interpolation("problem_3", [(2, 1), (3, 0), (4, 4)])
print(f"The approximated value is {solve_problem_1_3(4)}")

The approximated value is 6.499999999909355


![Graph of the interpolated function](./plots/problem_3_interpolated_func.png)
![Graph of the interpolated function's derivative](./plots/problem_3_interpolated_deriv.png)

---

За да намерим приближената стойност на първата производна в точка 4, нека разгледаме тройката $(3, 4, 5)$. При нея стъпката $h = 1$.

$$N'(f; 4) = \frac {f(5) - f(3)} {2} = \frac {3 - 0} {2} = \frac {3} {2}$$
$$R'(4) = -\frac {f'''(\xi)} {6} \implies |R'(4)| \le \frac {1} {60}$$

In [12]:
(_, solve_problem_1_4) = visualize_newton_interpolation("problem_4", [(3, 0), (4, 4), (5, 3)])
print(f"The approximated value is {solve_problem_1_4(4)}")

The approximated value is 1.4999999999654177


![Graph of the interpolated function](./plots/problem_4_interpolated_func.png)
![Graph of the interpolated function's derivative](./plots/problem_4_interpolated_deriv.png)

### Използвана литература

1. Вежди Хасанов (2019). Ръководство по Числени методи с MATLAB. Шумен. ISBN 978-619-201-310-3
2. [Divided differences](https://en.wikipedia.org/wiki/Divided_differences). Wikipedia. Посетен на 2022-12-08 (на английски)
3. [Numerical differentiation](https://en.wikipedia.org/wiki/Numerical_differentiation). Wikipedia. Посетен на 2022-12-01 (на английски)